# Игровая индустрия 2000-2013: тренды, платформы и RPG

- Автор: Заславский Данила
- Дата: 20.03.2025

### Цели и задачи проекта

Данный проект посвящён проведению исследования развития игровой индустрии в период с 2000 по 2013 год для написания аналитической статьи, которая привлечёт внимание любителей старых игр и вызовет интерес к игре "Секреты Темнолесья". Для этого будет необходимо ознакомиться и проанализоровать данные о продажах игр, жанрах, платформах, оценках пользователей и критиков. Основной акцент будет сделан на RPG-играх, чтобы подчеркнуть их роль в развитии индустрии и связать с современными трендами.

### Описание данных

В проекте будут использованы данные датасета `new_games.csv` с таким описанием:
- `Name` - название игры;
- `Platform` - название платформы;
- `Year of Release` - год выпуска игры;
- `Genre` - жанр игры;
- `NA sales` - продажи в Северной Америке (в миллионах проданных копий);
- `EU sales` - продажи в Европе (в миллионах проданных копий);
- `JP sales` - продажи в Японии (в миллионах проданных копий);
- `Other sales` - продажи в других странах (в миллионах проданных копий);
- `Critic Score` - оценка критиков (от 0 до 100);
- `User Score` - оценка пользователей (от 0 до 10);
- `Rating` - рейтинг организации ESRB.

### Содержимое проекта

[1. Загрузка данных и знакомство с ними](#chapter1)  
[2. Проверка ошибок в данных и их предобработка](#chapter2)    
&nbsp;&nbsp;[2.1. Названия, или метки, столбцов датафрейма](#chapter2_1)  
&nbsp;&nbsp;[2.2. Типы данных](#chapter2_2)  
&nbsp;&nbsp;[2.3. Наличия пропусков в данных](#chapter2_3)  
&nbsp;&nbsp;[2.4. Явные и неявные дубликаты в данных](#chapter2_4)  
[3. Фильтрация данных](#chapter3)  
[4. Категоризация данных](#chapter4)  
[5. Итоговый вывод](#chapter5)  



## 1. Загрузка данных и знакомство с ними
<a id='chapter1'></a>

- Загрузите необходимые библиотеки Python и данные датасета `/datasets/new_games.csv`.


In [4]:
# Импортируем библиотеку pandas
import pandas as pd

In [5]:
# Выгружаем данные из датасета new_games.csv в датафрейм games
games = pd.read_csv('https://code.s3.yandex.net/datasets/new_games.csv')


- Познакомимся с данными: выведем первые строки и результат метода `info()`.


In [7]:
# Выводим информацию о датафрейме
games.info()
#нахождение количества строк датафрейма до предобработки
len_start = len(games)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16956 entries, 0 to 16955
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Name             16954 non-null  object 
 1   Platform         16956 non-null  object 
 2   Year of Release  16681 non-null  float64
 3   Genre            16954 non-null  object 
 4   NA sales         16956 non-null  float64
 5   EU sales         16956 non-null  object 
 6   JP sales         16956 non-null  object 
 7   Other sales      16956 non-null  float64
 8   Critic Score     8242 non-null   float64
 9   User Score       10152 non-null  object 
 10  Rating           10085 non-null  object 
dtypes: float64(4), object(7)
memory usage: 1.4+ MB


In [8]:
# Выводим первые строки датафрейма на экран
games.head()

,Name,Platform,Year of Release,Genre,NA sales,EU sales,JP sales,Other sales,Critic Score,User Score,Rating
0,Wii Sports,Wii,2006.0,Sports,41.36,28.96,3.77,8.45,76.0,8,E
1,Super Mario Bros.,NES,1985.0,Platform,29.08,3.58,6.81,0.77,NaN,NaN,NaN
2,Mario Kart Wii,Wii,2008.0,Racing,15.68,12.76,3.79,3.29,82.0,8.3,E
3,Wii Sports Resort,Wii,2009.0,Sports,15.61,10.93,3.28,2.95,80.0,8,E
4,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,11.27,8.89,10.22,1.00,NaN,NaN,NaN


Датасет `new_games.csv` содержит 11 стобцов и 16956 строк, в которых представлена информация о продажах игр, их жанрах, платформах, оценках пользователей и критиков.

Изучим типы данных и их корректность:
- **Числовые значения с плавающей запятой (float64).** Четыре столбца представлены типом `float64`. Это `Year of Release`, `NA sales`, `Other sales`, `Critic Score`.   
`Year of Release`содержит информацию о дате выпуска игры, представленной только значением года. Для данного случая рекомендуется использовать тип `int16`, который облегчит выполнение вычислений. 
- **Строковые данные (object)**. Семь столбцов имеют тип данных `object`:
    - `Name`, `Platform` и `Genre` содержат строковую информацию (название игры, её платформа и жанр), что логично для текстовых данных. Здесь тип данных `object` подходит.
    - `EU sales`, `JP sales ` и `User Score` содержат информацию о количестве продаж в разных регионах и оценке пользователей. Для таких данных рекомендуется использовать тип `float64` для проведения арифметических операций и агрегирующих функций.
    - `Rating` содержит рейтинг организации ESRB, представленный одним из 7 обозначений. Рекомендуемый тип - `category`.
    
В датасете встречаются пропуски в 6 столбцах `Name`, `Year of Release`,`Genre`,`Critic Score`,`User Score`,`Rating`.   

Названия столбцов отражают содержимое столбцов, однако их можно нормализовать, приведя к общему оформлению **snake_case**.

---

## 2.  Проверка ошибок в данных и их предобработка
<a id='chapter2'></a>


### 2.1. Названия, или метки, столбцов датафрейма
<a id='chapter2_1'></a>

- Выведите на экран названия всех столбцов датафрейма и проверьте их стиль написания.
- Приведите все столбцы к стилю snake case. Названия должны быть в нижнем регистре, а вместо пробелов — подчёркивания.

In [9]:
games.columns

Index(['Name', 'Platform', 'Year of Release', 'Genre', 'NA sales', 'EU sales',
       'JP sales', 'Other sales', 'Critic Score', 'User Score', 'Rating'],
      dtype='object')

Приведём стиль написания названий столбцов к **snake_case**: 

In [10]:
games = games.rename(columns = { 'Name' : 'name',
                               'Platform' : 'platform',
                               'Year of Release' : 'year_of_release',
                               'Genre' : 'genre',
                               'NA sales' : 'na_sales',
                               'EU sales' : 'ea_sales',
                               'JP sales' : 'jp_sales',
                               'Other sales' : 'other_sales',
                               'Critic Score' : 'critic_score',
                               'User Score' : 'user_score',
                               'Rating' : 'rating'
                               })

### 2.2. Типы данных
<a id='chapter2_2'></a>

- Если встречаются некорректные типы данных, предположите их причины.
- При необходимости проведите преобразование типов данных. Помните, что столбцы с числовыми данными и пропусками нельзя преобразовать к типу `int64`. Сначала вам понадобится обработать пропуски, а затем преобразовать типы данных.

В датасете встречаются некорректные типы данных. Скорее всего, это связано с тем, что тип данных назначался автоматически. Проведём преобразование типов данных необходимым столбцам:  

In [11]:
#Приведём столбец year_of_release к типу int16, 
#заменив строковые значения на пропуски
games['year_of_release'] = pd.to_numeric(games['year_of_release'], errors = 'coerce')
games['year_of_release'] = games['year_of_release'].fillna(0).astype('int16')

In [12]:
#Приведём столбцы ea_sales, jp_sales и user_score к типу float64
#Случайные строковые значения заменим на пропуски
games['ea_sales'] = pd.to_numeric(games['ea_sales'], errors = 'coerce')
games['jp_sales'] = pd.to_numeric(games['jp_sales'], errors = 'coerce')
games['user_score'] = pd.to_numeric(games['user_score'], errors = 'coerce')

In [13]:
#Приведём столбец rating к типу category
games['rating']  = games['rating'].astype('category')
games.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16956 entries, 0 to 16955
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype   
---  ------           --------------  -----   
 0   name             16954 non-null  object  
 1   platform         16956 non-null  object  
 2   year_of_release  16956 non-null  int16   
 3   genre            16954 non-null  object  
 4   na_sales         16956 non-null  float64 
 5   ea_sales         16950 non-null  float64 
 6   jp_sales         16952 non-null  float64 
 7   other_sales      16956 non-null  float64 
 8   critic_score     8242 non-null   float64 
 9   user_score       7688 non-null   float64 
 10  rating           10085 non-null  category
dtypes: category(1), float64(6), int16(1), object(3)
memory usage: 1.2+ MB


### 2.3. Наличие пропусков в данных
<a id='chapter2_3'></a>

- Посчитаем количество пропусков в каждом столбце в абсолютных и относительных значениях.


In [14]:
#Посчитаем количество пропущенных строк в датафрейме
games.isna().sum()

name                  2
platform              0
year_of_release       0
genre                 2
na_sales              0
ea_sales              6
jp_sales              4
other_sales           0
critic_score       8714
user_score         9268
rating             6871
dtype: int64

In [15]:
#Посчитаем процент строк с пропусками
games.isna().mean() * 100

name                0.011795
platform            0.000000
year_of_release     0.000000
genre               0.011795
na_sales            0.000000
ea_sales            0.035386
jp_sales            0.023590
other_sales         0.000000
critic_score       51.391838
user_score         54.659118
rating             40.522529
dtype: float64

Датасет `new_games.csv` содержит 25140 пропущенных значений, среди них:
 - 2 пропуска в столбце `name`;
 - 273 пропуска в столбце `year_of_release`;
 - 2 пропуска в столбце `genre`;
 - 6 пропусков в столбце `ea_sales`;
 - 4 пропусков в столбце `jp_sales`;
 - 8714 пропусков в столбце `critic_score`;
 - 9268 пропусков в столбце `user_score`;
 - 6871 пропуск в столбце `rating` 
 
Пропуски в столбце `year_of_release` представлены нулями. Причинами возникновения пропусков в столбцах могут быть ошибки при сборе данных, отсутвие или потеря информации.     
Обработаем пропущенные значения:


Удалим 2 пропуска в столбце `name`, так как название игры - ключевая информация:

In [16]:
games = games.dropna(subset = ['name'])

273 пропуска в столбце `year_of_release`, представленные нулями, при фильтрации по годам уйдут, поэтому удалим их: 

In [17]:
games = games[games['year_of_release'] != 0]

 
2 пропуска в столбце `genre` удаляем, так как жанр - важная характеристика игры:

In [18]:
games = games.dropna(subset = ['genre'])

Пропуски в столбцах `ea_sales` и `jp_sales` заполним средним значением продаж для конкретной платформы и года выпуска:

In [19]:
#функция для заполнения пропусков средним значением по платформе и году выпуска
def calc_average(group):
    mean_value = group.mean()
    return group.fillna(mean_value)

#применяем функцию к столбцам
games['ea_sales'] = games.groupby(['platform','year_of_release'])['ea_sales'].apply(calc_average) 
games['jp_sales'] = games.groupby(['platform','year_of_release'])['jp_sales'].apply(calc_average)

Пропуски в столбцах `critic_score` и `user_score` заполним средним значением оценки критиков и пользователей

In [20]:
games['critic_score'] = games['critic_score'].fillna(games['critic_score'].mean())
games['user_score'] = games['user_score'].fillna(games['user_score'].mean())
games.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16679 entries, 0 to 16955
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype   
---  ------           --------------  -----   
 0   name             16679 non-null  object  
 1   platform         16679 non-null  object  
 2   year_of_release  16679 non-null  int16   
 3   genre            16679 non-null  object  
 4   na_sales         16679 non-null  float64 
 5   ea_sales         16679 non-null  float64 
 6   jp_sales         16679 non-null  float64 
 7   other_sales      16679 non-null  float64 
 8   critic_score     16679 non-null  float64 
 9   user_score       16679 non-null  float64 
 10  rating           9901 non-null   category
dtypes: category(1), float64(6), int16(1), object(3)
memory usage: 1.3+ MB


Заполним пропуски столбца `rating` значением-индикатором "Unknown", чтобы явно обозначить отсутсвие рейтинга 

In [21]:
#проверка уникальных значений перед заполнением пропусков индикатором
games['rating'].unique()

['E', NaN, 'M', 'T', 'E10+', 'K-A', 'AO', 'EC', 'RP']
Categories (8, object): ['E', 'M', 'T', 'E10+', 'K-A', 'AO', 'EC', 'RP']

Уникальные значения соответствуют градации рейтинга организации ESRB.

In [22]:
games['rating'] = games['rating'].astype('object')
games['rating'] = games['rating'].fillna('UNKNOWN')
games['rating'] = games['rating'].astype('category')                                         
games.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16679 entries, 0 to 16955
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype   
---  ------           --------------  -----   
 0   name             16679 non-null  object  
 1   platform         16679 non-null  object  
 2   year_of_release  16679 non-null  int16   
 3   genre            16679 non-null  object  
 4   na_sales         16679 non-null  float64 
 5   ea_sales         16679 non-null  float64 
 6   jp_sales         16679 non-null  float64 
 7   other_sales      16679 non-null  float64 
 8   critic_score     16679 non-null  float64 
 9   user_score       16679 non-null  float64 
 10  rating           16679 non-null  category
dtypes: category(1), float64(6), int16(1), object(3)
memory usage: 1.3+ MB


In [23]:
#проверка типа данных столбца `year_of_release`
games['year_of_release'].dtypes

dtype('int16')

In [24]:
#проверка типа данных столбца `critic_score`
games['critic_score'] = games['critic_score'].astype('int16')
games['critic_score'].dtypes

dtype('int16')

Проверка на наличие пропусков показала наличие в датасете `new_games.csv` 25140 пропущенных значений, среди них:
 - 2 пропуска в столбце `name`;
 - 273 пропуска в столбце `year_of_release`;
 - 2 пропуска в столбце `genre`;
 - 6 пропусков в столбце `ea_sales`;
 - 4 пропусков в столбце `jp_sales`;
 - 8714 пропусков в столбце `critic_score`;
 - 9268 пропусков в столбце `user_score`;
 - 6871 пропуск в столбце `rating`.  

Была проведена обработка полученных значений, так:     
 - Пропуски в столбцах `name`, `year_of_release`, `genre` были удалены; 
 - пропуски в столбцах  `ea_sales`, `jp_sales`, `critic_score`, `user_score` были заменены на средние значения;
 - пропуски в столбце `rating` были заменены на индикатор `UNKNOWN`.  
   
Таким образом, в датасете осталось 16679 значений. 

### 2.4. Явные и неявные дубликаты в данных
<a id='chapter2_4'></a>

- Изучите уникальные значения в категориальных данных, например с названиями жанра игры, платформы, рейтинга и года выпуска. Проверьте, встречаются ли среди данных неявные дубликаты, связанные с опечатками или разным способом написания.
- При необходимости проведите нормализацию данных с текстовыми значениями. Названия или жанры игр можно привести к нижнему регистру, а названия рейтинга — к верхнему.

Рассмотрим уникальные значения в каждом категориальном столбце. Выведем столбец `genre` 

In [25]:
games['genre'].unique()

array(['Sports', 'Platform', 'Racing', 'Role-Playing', 'Puzzle', 'Misc',
       'Shooter', 'Simulation', 'Action', 'Fighting', 'Adventure',
       'Strategy', 'MISC', 'ROLE-PLAYING', 'RACING', 'ACTION', 'SHOOTER',
       'FIGHTING', 'SPORTS', 'PLATFORM', 'ADVENTURE', 'SIMULATION',
       'PUZZLE', 'STRATEGY'], dtype=object)

В столбце встречаются неявные дубликаты, например, 'Misc' и 'MISC'. Требуется нормализация данных.  
Рассмотрим столбец `platform`

In [26]:
games['platform'].unique()

array(['Wii', 'NES', 'GB', 'DS', 'X360', 'PS3', 'PS2', 'SNES', 'GBA',
       'PS4', '3DS', 'N64', 'PS', 'XB', 'PC', '2600', 'PSP', 'XOne',
       'WiiU', 'GC', 'GEN', 'DC', 'PSV', 'SAT', 'SCD', 'WS', 'NG', 'TG16',
       '3DO', 'GG', 'PCFX'], dtype=object)

Данные о платформе уникальны, нормализация не требуется.  
Рассмотрим столбец `rating`

In [27]:
games['rating'].unique()

['E', 'UNKNOWN', 'M', 'T', 'E10+', 'K-A', 'AO', 'EC', 'RP']
Categories (9, object): ['E', 'UNKNOWN', 'M', 'T', ..., 'K-A', 'AO', 'EC', 'RP']

Данные о рейтинге ESRB уникальны, нормализация не требуется.  
Рассмотрим столбец `year_of_release`

In [28]:
games['year_of_release'].unique()

array([2006, 1985, 2008, 2009, 1996, 1989, 1984, 2005, 1999, 2007, 2010,
       2013, 2004, 1990, 1988, 2002, 2001, 2011, 1998, 2015, 2012, 2014,
       1992, 1997, 1993, 1994, 1982, 2016, 2003, 1986, 2000, 1995, 1991,
       1981, 1987, 1980, 1983], dtype=int16)

Данные о годах релиза уникальны, нормализация не требуется.  
Проведем нормализацию данных в столбцах `name` и `genre`

In [29]:
#Приводим столбцы `name` и `genre` к нижнему регистру
games['name'] = games['name'].str.lower()
games['genre'] = games['genre'].str.lower()

- После того как нормализуете данные и устраните неявные дубликаты, проверьте наличие явных дубликатов в данных.

Проверим датасет на наличие явных дубликатов.

In [30]:
games.duplicated().sum()

235

В датасете находится 235 явных дубликатов. Воспользуемся методом `drop_duplicates`:

In [31]:
games = games.drop_duplicates()
games.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16444 entries, 0 to 16955
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype   
---  ------           --------------  -----   
 0   name             16444 non-null  object  
 1   platform         16444 non-null  object  
 2   year_of_release  16444 non-null  int16   
 3   genre            16444 non-null  object  
 4   na_sales         16444 non-null  float64 
 5   ea_sales         16444 non-null  float64 
 6   jp_sales         16444 non-null  float64 
 7   other_sales      16444 non-null  float64 
 8   critic_score     16444 non-null  int16   
 9   user_score       16444 non-null  float64 
 10  rating           16444 non-null  category
dtypes: category(1), float64(5), int16(2), object(3)
memory usage: 1.2+ MB


- Напишите промежуточный вывод: укажите количество найденных дубликатов и действия по их обработке.

После проверки столбцов на неявные дубликаты было обнаружено их наличие в столбце `genre`. Для устранения этой проблемы столбец был приведён к нижнему регистру. В остальных столбцах неявных дубликатов обнаружено не было.  
После нормализации данных проверка на явные дубликаты показала наличие 235 явных дубликатов. Для их устранения был использован метод `drop_duplicates`. После удаления дубликатов количество строк в датасете стало равных 16444.

- В процессе подготовки данных вы могли что-либо удалять, например строки с пропусками или ошибками, дубликаты и прочее. В этом случае посчитайте количество удалённых строк в абсолютном и относительном значениях.

Ранее мы посчитали количество строк в датасете в переменной **len_start**. Найдём количество строк в обработанном датафрейме, а также количество удаленных строк в абсолютном относительном значениях.

In [32]:
#находим количество строк после предобработки
len_finish = len(games)

#находим количество удалённых строк
count_del_rows = len_start - len_finish
print(f'Количество удалённых строк: {count_del_rows}')
#находим количество удалённых строк в относительном значении
relative_del_rows = round(count_del_rows / len_start,2) * 100
print(f'Относительное количество удалённых строк: {relative_del_rows} %')

Количество удалённых строк: 512
Относительное количество удалённых строк: 3.0 %


- После проведения предобработки данных напишите общий промежуточный вывод.

Перед началом работы датасет содержал 16 956 строк и 11 столбцов. Были обнаружены пропуски, некорректные типы данных и дубликаты.  
Названия столбцов были приведены к `snake_case`. Были исправлены типы данных у некоторых столбцов: `year_of_release`,`critic_score` → **int16**, `ea_sales`, `jp_sales`, `user_score` → **float64**, `rating` → **category**.  
Удалены строки с пропусками в `name`(2), `year_of_release`(273), `genre`(2).  
Пропуски в `rating` заменены на 'UNKNOWN'.  
Остальные пропуски заполнены средними значениями.  
Было удалено 235 явных дубликатов.  
Таким образом, было удалено 512 строк, составляющие 3% от исходного датафрейма. Остальные 16 444 строки очищены и готовы для дальнейшей работы.

---

## 3. Фильтрация данных
<a id='chapter3'></a>

Коллеги хотят изучить историю продаж игр в начале XXI века, и их интересует период с 2000 по 2013 год включительно. Отберите данные по этому показателю. Сохраните новый срез данных в отдельном датафрейме, например `df_actual`.

In [33]:
#фильтруем датафрейм по дате
df_actual = games[(games['year_of_release'] >= 2000) & (games['year_of_release'] <= 2013)]                                                      

In [34]:
#выводим информацию о новом датафрейме
df_actual.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12781 entries, 0 to 16954
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype   
---  ------           --------------  -----   
 0   name             12781 non-null  object  
 1   platform         12781 non-null  object  
 2   year_of_release  12781 non-null  int16   
 3   genre            12781 non-null  object  
 4   na_sales         12781 non-null  float64 
 5   ea_sales         12781 non-null  float64 
 6   jp_sales         12781 non-null  float64 
 7   other_sales      12781 non-null  float64 
 8   critic_score     12781 non-null  int16   
 9   user_score       12781 non-null  float64 
 10  rating           12781 non-null  category
dtypes: category(1), float64(5), int16(2), object(3)
memory usage: 961.4+ KB


После фильтрации по дате количество строк в датафрейме сократилось с **16 444** до **12 781**.


---

## 4. Категоризация данных
<a id='chapter4'></a>
    
Проведите категоризацию данных:
- Разделите все игры по оценкам пользователей и выделите такие категории: высокая оценка (от 8 до 10 включительно), средняя оценка (от 3 до 8, не включая правую границу интервала) и низкая оценка (от 0 до 3, не включая правую границу интервала).

In [35]:
df_actual['user_score_category'] = pd.cut(df_actual['user_score'], bins = [0, 3, 8, 10], 
                                      labels = ['низкая оценка', 'средняя оценка', 'высокая оценка'],
                                      right = False)

/tmp/ipykernel_31/3712519711.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_actual['user_score_category'] = pd.cut(df_actual['user_score'], bins = [0, 3, 8, 10],


- Разделите все игры по оценкам критиков и выделите такие категории: высокая оценка (от 80 до 100 включительно), средняя оценка (от 30 до 80, не включая правую границу интервала) и низкая оценка (от 0 до 30, не включая правую границу интервала).

In [36]:
df_actual['critic_score_category'] = pd.cut(df_actual['critic_score'], bins = [0, 30, 80, 100], 
                                      labels = ['низкая оценка', 'средняя оценка', 'высокая оценка'],
                                      right = False)

/tmp/ipykernel_31/2813004753.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_actual['critic_score_category'] = pd.cut(df_actual['critic_score'], bins = [0, 30, 80, 100],


- После категоризации данных проверьте результат: сгруппируйте данные по выделенным категориям и посчитайте количество игр в каждой категории.

In [37]:
#считаем количество игр по категориям оценок пользователей
grouped_user_score = df_actual.groupby('user_score_category').size()
print(grouped_user_score)

user_score_category
низкая оценка       116
средняя оценка    10379
высокая оценка     2286
dtype: int64


In [38]:
#считаем количество игр по категориям оценок критиков
grouped_critic_score = df_actual['critic_score_category'].value_counts()
print(grouped_critic_score)

средняя оценка    11034
высокая оценка     1692
низкая оценка        55
Name: critic_score_category, dtype: int64


In [39]:
#функция для категоризации оценок критиков
def categorize_critic_score(score):
    if 0 <= score < 30:
        return 'низкая оценка'
    elif 30 <= score < 80:
        return 'средняя оценка'
    elif 80 <= score <= 100:
        return 'высокая оценка'
    else:
        return None

#функция для категоризации оценок пользователей
def categorize_user_score(score):
    if 0 <= score < 3:
        return 'низкая оценка'
    elif 3 <= score < 8:
        return 'средняя оценка'
    elif 8 <= score <= 10:
        return 'высокая оценка'
    else:
        return None

In [40]:
#Применяем функцию к столбцам `critic_score` и `user_score`
df_actual['critic_score_category2'] = df_actual['critic_score'].apply(categorize_critic_score)
df_actual['user_score_category2'] = df_actual['user_score'].apply(categorize_user_score)
df_actual.head(10)

/tmp/ipykernel_31/1104004688.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_actual['critic_score_category2'] = df_actual['critic_score'].apply(categorize_critic_score)
/tmp/ipykernel_31/1104004688.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_actual['user_score_category2'] = df_actual['user_score'].apply(categorize_user_score)


,name,platform,year_of_release,genre,na_sales,ea_sales,jp_sales,other_sales,critic_score,user_score,rating,user_score_category,critic_score_category,critic_score_category2,user_score_category2
0,wii sports,Wii,2006,sports,41.36,28.96,3.77,8.45,76,8.000000,E,высокая оценка,средняя оценка,средняя оценка,высокая оценка
2,mario kart wii,Wii,2008,racing,15.68,12.76,3.79,3.29,82,8.300000,E,высокая оценка,высокая оценка,высокая оценка,высокая оценка
3,wii sports resort,Wii,2009,sports,15.61,10.93,3.28,2.95,80,8.000000,E,высокая оценка,высокая оценка,высокая оценка,высокая оценка
6,new super mario bros.,DS,2006,platform,11.28,9.14,6.50,2.88,89,8.500000,E,высокая оценка,высокая оценка,высокая оценка,высокая оценка
7,wii play,Wii,2006,misc,13.96,9.18,2.93,2.84,58,6.600000,E,средняя оценка,средняя оценка,средняя оценка,средняя оценка
8,new super mario bros. wii,Wii,2009,platform,14.44,6.94,4.70,2.24,87,8.400000,E,высокая оценка,высокая оценка,высокая оценка,высокая оценка
10,nintendogs,DS,2005,simulation,9.05,10.95,1.93,2.74,68,7.124438,UNKNOWN,средняя оценка,средняя оценка,средняя оценка,средняя оценка
11,mario kart ds,DS,2005,racing,9.71,7.47,4.13,1.90,91,8.600000,E,высокая оценка,высокая оценка,высокая оценка,высокая оценка
13,wii fit,Wii,2007,sports,8.92,8.03,3.60,2.15,80,7.700000,E,средняя оценка,высокая оценка,высокая оценка,средняя оценка
14,kinect adventures!,X360,2010,misc,15.00,4.89,0.24,1.69,61,6.300000,E,средняя оценка,средняя оценка,средняя оценка,средняя оценка


- Выделите топ-7 платформ по количеству игр, выпущенных за весь актуальный период.

In [41]:
grouped_year = df_actual['platform'].value_counts()
grouped_year.head(7)

PS2     2127
DS      2120
Wii     1275
PSP     1180
X360    1121
PS3     1087
GBA      811
Name: platform, dtype: int64

Теперь датафрейм содержит столбец `grouped_user_score`, содержащий информацию о категории оценки пользователей и  `grouped_critic_score`, содержащий информацию о категории оценки критиков.  
Самыми популярными платформами за актуальный период являются: PS2(2127), DS(2120), Wii(1275), PSP(1180), X360(1121), PS3(1087) и GBA(811).

---

## 5. Итоговый вывод
<a id='chapter5'></a>

В конце напишите основной вывод и отразите, какую работу проделали. Не забудьте указать описание среза данных и новых полей, которые добавили в исходный датасет.

Были загружены данные датасета `new_games.csv`. Они содержали 11 столбцов и 16 956 строк, в которым была информация о продажах игр, жанрах, платформах, оценках пользователей и критиков. При первичном знакомстве с данными и их предобработкой были получены следующие результаты:
 - В восьми столбцах (`name`, `year_of_release`, `genre`, `ea_sales`, `jp_sales`, `critic_score`, `user_score`, `rating`) были обнаружены пропущенные значения. Максимальное значение пропущенных данных в столбце `user_score` - 9268.  
 Пропуски в столбцах `name`, `year_of_release`, `genre` были удалены, в столбцах  `ea_sales`, `jp_sales`, `critic_score`, `user_score` были заменены на средние значения, а в столбце `rating` - на `UNKNOWN`.
 - Для оптимизации работы с данными в датафрейме были произведены следующие изменения типов данных:
     - `year_of_release`, `critic_score`: тип данных изменён с `float64` на `int16`;
     - `ea_sales`, `jp_sales` и `user_score`: тип данных изменён с `object` на `float64`;
     - `rating`: тип данных изменён с `object` на `category`.
 - Для дополнительной работы с данными были созданы столбцы:
     - `user_score_category`: столбец, содержащий три категории оценки пользователей;
     - `critic_score_category`: столбец, содержащий три категории оценки критиков;
 - Датафрейм был отфильтрован по дате по диапазону от 2000 до 2013 включительно, чтобы выделить историю продаж игр в начале XXI века.
 - После работы с дубликатами и пропусками было удалено 512 строк, составляющих 3% исходного датафрейма. Итоговый датафрейм содержит 16444 строки.  
Основные наблюдения по данных после предобработки:
 - оценки пользователей: большинство игр получили средние оценки (3-8), что говорит о том, что пользователи чаще ставят средние баллы. Высокие оценки (8-10) получили 2286 игр, а низкие (0-3) - 116.
 - оценки критиков: критики чаще ставят средние оценки (30-80). Высоких оценок - 1692, низких(0-30) - 55.
 - популярные платформы: лидерами по количеству выпущенных игр являются PS2 (2127) и DS(2120), что подтверждает их популярность в начале XXI века.
     



<div class="alert alert-success">
<h2> Комментарий ревьюера <a class="tocSkip"> </h2>

<b>Все отлично!👍:  </b> 

Молодец, что не забываешь написать итоговый вывод по проекту. И отлично, что  пишешь выводы по предобработке — это хорошая практика, используй пожалуйста ее в следующих проектах, когда будешь готовить данные 

</div>   
    
    
<div style="border:solid Chocolate 2px; padding: 40px">


<h2> Итоговый комментарий ревьюера (общий вывод по проекту)  <a class="tocSkip"> </h2>    
    
  
У меня сложилось хорошее впечатление о проекте. Молодец! Осмысленная аналитика и дельная работа - многое удалось как надо)
    
Отмечу отдельные положительные моменты проекта🙂:
    
- Была проделана значительная работа по обработке и анализу данных! 
- Твой вывод чётко отражает проделанные шаги, что важно для демонстрации результатов исследования.   
    
Есть несколько моментов всего, на которые стоит ещё раз взглянуть, я указал их в моих комментариях по ходу проекта.
    
Предлагаю тебе доработать проект по моим комментариям, чтобы довести его до совершенства.   
    

 
</div>


<div style="border:solid Chocolate 2px; padding: 40px">


<h2>  Итоговый комментарий ревьюера 2 (итоговый вывод по проекту) <a class="tocSkip"> </h2>    
    
      
  
Теперь почти идеально, молодец! Принимаю твой проект)
    
Могу порекомендовать тебе отличную книгу про Python:
    
https://monster-book.com/avtomatizaciya-zadach-s-python    
    
По визуализации могу посоветовать следующие материалы:
    
1. https://github.com/esokolov/ml-course-hse/blob/master/2022-fall/seminars/sem02-charts.ipynb
    
2. https://github.com/Palladain/Deep_Python/blob/main/Lectures/Lecture_3/Lecture_3_Python.ipynb    
    
Советую тебе обратить внимание на библиотеки `sweetviz` и `ydata_profiling`, которые помогут в проведении более тщательного EDA анализа. Данные библиотеки хороши для максимизации комфорта презентации результатов анализа бизнес-пользователям. 
    
Очень просты в использоовании, на вход кладется датафрейм: `ydata_profiling.ProfileReport(df)`
    
Цель нашего проекта достигнута. Рад был тебе помочь)
    
Удачи тебе в следующем проекте!
    
**Буду очень благодарен, если ты поставишь мне положительную оценку за проект в форме на странице курса, это важно для меня. Достаточно поставить оценку, а комментарий можешь написать по желанию:)**     
    
</div>    
